In [3]:
import pandas as pd
import numpy as np
import re
import torch
import json
from tqdm.notebook import tqdm
# from torch.utils.data import Dataset, DataLoader
# from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.model_selection import train_test_split

In [4]:
import sys

sys.path.append('../utils')
from evaluator import Evaluator
from json_format import process_json, unprocess_json

In [5]:
train = pd.read_csv('../data/train_9k_valid.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)
val_set = pd.read_csv('../data/val_set_300_sb_valid.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)
manual_test = pd.read_csv('../data/manual_test_100.csv', index_col=0, converters={'json': json.loads}).sample(frac=1, random_state=42)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("ai-forever/ruT5-large")
model = AutoModelForSeq2SeqLM.from_pretrained("ai-forever/ruT5-large")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
tokenizer.add_tokens(["<BOB>", "<EOB>", "<BOT>", "<EOT>", "<BOP>", "<EOP>", "<BOC1>", "<EOC1>", "<BOC2>", "<EOC2>"])
model.resize_token_embeddings(len(tokenizer))


assert train.loc[train.index[0], 'json'] == unprocess_json(process_json(train.loc[train.index[0], 'json']))

In [8]:
from datasets import Dataset

ads_dataset = Dataset.from_pandas(train[["Text", "json"]])
ads_dataset = ads_dataset.train_test_split(test_size=0.025, seed=42)
ads_dataset = ads_dataset.flatten()
ads_dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 8590
    })
    test: Dataset({
        features: ['Text', 'json', '__index_level_0__'],
        num_rows: 221
    })
})

In [9]:
def preprocess_function(examples):
    inputs = [text for text in examples["Text"]]
    # targets = ['' for bundles in examples["json"]]
    targets = [process_json(bundles) for bundles in examples["json"]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

ads = ads_dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=ads_dataset["train"].column_names
)
ads = ads.flatten()

# ads_test = ads_test_dataset.map(
#     preprocess_function,
#     batched=True,
#     num_proc=4,
#     remove_columns=ads_test_dataset.column_names
# )
# ads_test = ads_test.flatten()


Map (num_proc=4):   0%|          | 0/8590 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/221 [00:00<?, ? examples/s]

In [10]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

In [11]:
from transformers import DataCollatorForSeq2Seq

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [12]:
from IPython.display import clear_output

class MetricComputer:
  def __init__(self):
    self.generations = []

  def __call__(self, eval_preds):
    ev = Evaluator(val_set, model=model, tokenizer=tokenizer)
    stats = ev.calc_bleu_batched()
    self.generations.append(ev.generate_samples_batched(count=20))
    # clear_output()
    return stats

In [13]:
n_epochs = 13

training_args = Seq2SeqTrainingArguments(
    output_dir="ruT5-large",
    # overwrite_output_dir=True,
    evaluation_strategy="steps",
    eval_steps=2000,
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=n_epochs,
    # predict_with_generate=True,
    generation_max_length=128,
    # fp16=True,
    lr_scheduler_type="cosine",
    group_by_length=False,
    warmup_steps=3,
)

mc = MetricComputer()
empty_dataset = Dataset.from_dict({"Text": [], "json": []})
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=ads["train"],
    eval_dataset=ads["test"],
    # eval_dataset=empty_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=mc,
)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/training_args.py:1493: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
trainer.train()

Step,Training Loss,Validation Loss,Bep-sb,Bep-multi,Ta-bleu-sb,Ta-chrf-sb,Ta-chrf-multi,Eb-ind,Mb-ind,Bleu-classic,Chrf-classic,Chrf-classic-multi,Bleu Old,Failed Ratio
2000,0.377200,0.305604,0.683618,0.640456,38.850878,78.115046,76.818380,0.018000,0.088000,51.863405,74.148369,72.127259,51.863405,0.000000
4000,0.232100,0.291323,0.707053,0.673667,38.830676,77.388651,76.993098,0.046000,0.050000,53.078581,74.217138,73.662266,53.078581,0.000000
6000,0.143800,0.303444,0.711244,0.679611,38.979014,78.373558,78.257050,0.024000,0.060000,48.947404,74.780289,75.539022,48.947404,0.000000
8000,0.095000,0.340101,0.719231,0.689527,40.500965,79.188329,79.033437,0.034000,0.052000,53.108882,75.880792,76.796622,53.108882,0.000000
10000,0.068900,0.357821,0.718127,0.694664,42.153629,79.283929,79.510316,0.024000,0.056000,51.530157,76.268901,77.689921,51.530157,0.000000
12000,0.055100,0.366410,0.718941,0.693127,41.766630,79.345507,79.448319,0.020000,0.058000,54.515328,76.545685,77.694866,54.515328,0.002000


/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/home/vlad/bralex/.venv/lib/python3.10/site-packa

<BOT> sony 3.5-5.6/16-50 oss with 150<EOP><BOC1> 1<EOC1><BOC2> GEL<EOC2>


TrainOutput(global_step=13962, training_loss=0.1857306680268125, metrics={'train_runtime': 10185.5607, 'train_samples_per_second': 10.964, 'train_steps_per_second': 1.371, 'total_flos': 5.2547879835648e+16, 'train_loss': 0.1857306680268125, 'epoch': 13.0})

In [13]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [14]:
output_dir = "ruT5-large-fp32-trained"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

('ruT5-large-fp32-trained/tokenizer_config.json',
 'ruT5-large-fp32-trained/special_tokens_map.json',
 'ruT5-large-fp32-trained/spiece.model',
 'ruT5-large-fp32-trained/added_tokens.json',
 'ruT5-large-fp32-trained/tokenizer.json')

In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_path = './ruT5-large-fp32-trained'
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
mc(1)

/home/vlad/bralex/.venv/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


{'BEP-sb': 0.7152080159361315,
 'BEP-multi': 0.6922981444121631,
 'TA-BLEU-sb': 41.425411733011856,
 'TA-CHRF-sb': 78.93872567666078,
 'TA-CHRF-multi': 79.21263143493536,
 'EB-ind': 0.02,
 'MB-ind': 0.056,
 'BLEU-classic': 52.1340390149972,
 'CHRF-classic': 76.06866977611242,
 'CHRF-classic-multi': 77.38755359181785,
 'bleu_old': 52.1340390149972,
 'failed_ratio': 0.0}

In [15]:
ev = Evaluator(manual_test, model=model, tokenizer=tokenizer, batch_size=32)
output = ev.generate_samples_batched()
df = pd.DataFrame([{'id': i, 'json': json.dumps(v, indent=4, ensure_ascii=False)} for i, v in output.items()])
df.to_csv('manual_test_outputs_fp32.csv', index=False)

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained(output_dir)
# model = AutoModelForSeq2SeqLM.from_pretrained(output_dir).to('cuda')

In [ ]:
# distill_data = pd.read_csv('../data/distill_data.csv', index_col=0)
# distill_data.head()

In [ ]:
# ev = Evaluator(distill_data, model, tokenizer)
# output = ev.generate_samples_batched(batch_size=256)

In [ ]:
# distill_data['json'] = pd.Series(output)

In [ ]:
# distill_data.to_csv('../data/distill_100k.csv')